In [1]:
# from base64 import b64encode, b64decode
# from binascii import hexlify, unhexlify, a2b_base64, b2a_base64, a2b_hex, b2a_hex

# The Matasano Crypto Challenges
http://cryptopals.com/

##1.1 Convert hex to base64
The string:  
>49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d

Should produce:  
>SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

So go ahead and make that happen. You'll need to use this code for the rest of the exercises.  

####Cryptopals Rule
    Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

In [2]:
b64Lookup = {
    'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,
    'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'a':26,'b':27,'c':28,'d':29,'e':30,'f':31,'g':32,
    'h':33,'i':34,'j':35,'k':36,'l':37,'m':38,'n':39,'o':40,'p':41,'q':42,'r':43,'s':44,'t':45,'u':46,'v':47,'w':48,
    'x':49,'y':50,'z':51,'0':52,'1':53,'2':54,'3':55,'4':56,'5':57,'6':58,'7':59,'8':60,'9':61,'+':62,'/':63,'=':64
}
def toBase64(byteArr):
    # see https://www.nczonline.net/blog/2009/12/08/computer-science-in-javascript-base64-encoding/
    # and https://en.wikipedia.org/wiki/Base64
    b64Dict = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/="
    res = ''
    for i in range(0,len(byteArr),3):
        b = (byteArr[i] & 0xfc)>>2                  # ******** & 11111100
        res += b64Dict[b]
        b = (byteArr[i] & 0x03)<<4                  # ******** & 00000011
        if (i+1 < len(byteArr)):
            b |= (byteArr[i+1] & 0xf0)>>4           # ******** & 11110000
            res += b64Dict[b]
            b = (byteArr[i+1] & 0x0f)<<2            # ******** & 00001111
            if i+1 < len(byteArr):
                b |= (byteArr[i+2] & 0xc0)>>6       # ******** & 11000000
                res += b64Dict[b]
                b = byteArr[i+2] & 0x3f             # ******** & 00111111
                res += b64Dict[b]
            else:
                res += b64Dict[b]
                res += '='
        else:
            res += b64Dict[(byteArr[i] & 0x03)<<4]    # ******** & 00000011
            res += '=='
    return res

def fromBase64(b64Str): # returns byte array
    # see: https://en.wikipedia.org/wiki/Base64
    if len(b64Str) % 4 != 0: raise Exception("Invalid base64 input")
    res = []
    b=[0,0,0,0]
    def toByte(val):
        if val > 255:
            val -= 256
            return toByte(val)
        else: return val
    for i in range(0,len(b64Str),4):
        for j in range(4):
            b[j]=b64Lookup[b64Str[i+j]]
        res.append(toByte((b[0]<<2) | (b[1]>>4)))
        if (b[2] < 64):
            res.append(toByte((b[1]<<4) | (b[2]>>2)))
            if b[3] < 64:
                res.append(toByte((b[2]<<6) | b[3]))
    return bytearray(res)

def fromHex(hexStr):
    if len(hexStr)%2 != 0: raise Exception("Invalid hex input. Expected even-number length.")
    hexLookup = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
               'a':10,'A':10,'b':11,'B':11,'c':12,'C':12,'d':13,'D':13,'e':14,'E':14,'f':15,'F':15}
    byteArr = []
    for i in range(0,len(hexStr),2):
        byteArr.append(16*hexLookup[hexStr[i]]+hexLookup[hexStr[i+1]])
    return bytearray(byteArr)

def toHex(byteArr):
    hexDict = '0123456789abcdef'
    res = ''
    for b in byteArr:
        res += hexDict[b / 16] + hexDict[b % 16]
    return res

hexStr = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
expected = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

b64Str = toBase64(fromHex(hexStr))

print "b64 encoded: "+b64Str

print "Expected:    "+expected
print "b64 encoding passed: %s\n"%(b64Str==expected)

# test toHex function
decodedBytes = fromBase64(b64Str)
reencodedHex = toHex(decodedBytes)
print "hex encoded: "+reencodedHex
print "Expected:    "+hexStr
print "hex encoding passed: %s" %(reencodedHex==hexStr)

b64 encoded: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
Expected:    SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
b64 encoding passed: True

hex encoded: 49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
Expected:    49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
hex encoding passed: True


##1.2 Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

>1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

>686974207468652062756c6c277320657965

... should produce:

>746865206b696420646f6e277420706c6179


In [3]:
def xorCipher(byteArr1, byteArr2): # returns bytearray
    byteArr2 = byteArr2*((len(byteArr1)/len(byteArr2))+1)
    res = []
    for i in range(len(byteArr1)):
        res.append(byteArr1[i] ^ byteArr2[i])
    return bytearray(res)

s1='1c0111001f010100061a024b53535009181c'
s2='686974207468652062756c6c277320657965'
expected='746865206b696420646f6e277420706c6179'

xored = xorCipher(fromHex(s1),fromHex(s2))
xoredHex = toHex(xored)
print "XORed:    "+xoredHex
print "Expected: "+expected
print "Passed: %s" % (xoredHex == expected)

XORed:    746865206b696420646f6e277420706c6179
Expected: 746865206b696420646f6e277420706c6179
Passed: True


##1.3 Single-byte XOR cipher
The hex encoded string:
>1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

####Achievement Unlocked
    You now have our permission to make "ETAOIN SHRDLU" jokes on Twitter.

In [4]:
def calcScore(byteArr):
    # English letter frequency table from http://www.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
    englishLetterFrequency={
        'E':12.02, 'T':9.10, 'A':8.12, 'O':7.68, 'I':7.31, 'N':6.95, 'S':6.28, 'R':6.02, 'H':5.92, 
        'D':4.32, 'L':3.98, 'U':2.88, 'C':2.71, 'M':2.61, 'F':2.30, 'Y':2.11, 'W':2.09, 'G':2.03, 
        'P':1.82, 'B':1.49, 'V':1.11, 'K':0.69, 'X':0.17, 'Q':0.11, 'J':0.10, 'Z':0.07,
        ' ':5
    }
    score = 0
    for b in byteArr:
        letter = chr(b).upper()
        if letter in englishLetterFrequency:
            score += englishLetterFrequency[letter]
    return score

def crackSingleCharXorCipher(byteArr):
    scores = []
    for i in range(256):
        res = xorCipher(byteArr, [i])
        scores.append((i,calcScore(res),bytearray(res)))

    scores = sorted(scores, key=lambda x:x[1], reverse=True)
    return scores[0]

hexStr = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
res = crackSingleCharXorCipher(fromHex(hexStr))

print "Key byte value: %i, ASCII char: %s, score: %s"%(res[0], bytearray([res[0]]), res[1])
print "Decoded: "+res[2]

Key byte value: 88, ASCII char: X, score: 169.35
Decoded: Cooking MC's like a pound of bacon


##1.4 Detect single-character XOR
One of the 60-character strings in [this file](/files/challenge4.txt) has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [5]:
with open('challenge4.txt') as file:
    txt = file.readlines()
    
res=[]
for lineNum, line in enumerate(txt):
    ba = fromHex(line.strip())
    res.append(tuple([lineNum])+crackSingleCharXorCipher(ba))
res = sorted(res,reverse=True,key=lambda x:x[2])[0]

print "Key byte value: %i, ASCII char: %s, score: %s"%(res[1], bytearray([res[1]]), res[2])
print "Encrypted line number: %i"%res[0]
print "Decoded: "+res[3]

Key byte value: 53, ASCII char: 5, score: 165.46
Encrypted line number: 170
Decoded: Now that the party is jumping



##1.5 Implement repeating-key XOR
Here is the opening stanza of an important work of the English language:

    Burning 'em, if you ain't quick and nimble
    I go crazy when I hear a cymbal

Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

    0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272
    a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f
Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.

In [6]:
plaintext = "Burning 'em, if you ain't quick and nimble\n"+\
            "I go crazy when I hear a cymbal"

expectedCipher = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272"+\
                "a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"
    
    
encryptedByteArr = xorCipher(bytearray(plaintext),bytearray("ICE"))
print toHex(encryptedByteArr)
print "Passed: %s"%(toHex(encryptedByteArr)==expectedCipher)

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f
Passed: True


##1.6 Break repeating-key XOR
####It is officially on, now.
>This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.

There's a file [here](/files/challenge6.txt). It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
1. Write a function to compute the edit distance/Hamming distance between two strings. *The Hamming distance is just the number of differing bits.* The distance between:
        this is a test
    and
        wokka wokka!!!

    is **37**. Make sure your code agrees before you proceed.
    
1. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
1. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
1. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
1. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
1. Solve each block as if it was single-character XOR. You already have code to do this.
1. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.

#####No, that's not a mistake.
>We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.

In [7]:
def bitDiff(b1,b2):
    res=0
    for i in range(8):
        res += abs( ( b1 >> i & 1 ) - ( b2 >> i & 1 ) )
    return res

def hammingDistance(byteArr1,byteArr2):
    if (len(byteArr1)>len(byteArr2)): # byteArr1 should be the shortest string
        tmp = byteArr1
        byteArr1 = byteArr2
        byteArr2 = tmp
    res=abs(len(byteArr1)-len(byteArr2))*8 # start with diff = 8 X difference in length
    for i,b in enumerate(byteArr1):
        res+=bitDiff(b,byteArr2[i])
    return res
    
print hammingDistance(bytearray("this is a test"), bytearray("wokka wokka!!!"))

with open("challenge6.txt") as file:
    encoded = fromBase64(''.join([line.strip() for line in file.readlines()]))

# encoded = bytearray([])
# with open("challenge6.txt") as file:
#     for line in file.readlines():
#         encoded += fromBase64(line.strip())

# print encoded

# Determine key size
dists=[]
for keysize in range(2,51):
    block0 = encoded[0:keysize]
    block1 = encoded[keysize:keysize*2]
    block2 = encoded[keysize*2:keysize*3]
    block3 = encoded[keysize*3:keysize*4]
    hammingDist = hammingDistance(block0, block1)
    hammingDist += hammingDistance(block0, block2)
    hammingDist += hammingDistance(block0, block3)
    hammingDist += hammingDistance(block1, block2)
    hammingDist += hammingDistance(block1, block3)
    hammingDist += hammingDistance(block2, block3)
    block0 = encoded[keysize*4:keysize*5]
    block1 = encoded[keysize*5:keysize*6]
    block2 = encoded[keysize*6:keysize*7]
    block3 = encoded[keysize*7:keysize*8]
    hammingDist = hammingDistance(block0, block1)
    hammingDist += hammingDistance(block0, block2)
    hammingDist += hammingDistance(block0, block3)
    hammingDist += hammingDistance(block1, block2)
    hammingDist += hammingDistance(block1, block3)
    hammingDist += hammingDistance(block2, block3)
    block0 = encoded[keysize*8:keysize*9]
    block1 = encoded[keysize*9:keysize*10]
    block2 = encoded[keysize*10:keysize*11]
    block3 = encoded[keysize*11:keysize*12]
    hammingDist = hammingDistance(block0, block1)
    hammingDist += hammingDistance(block0, block2)
    hammingDist += hammingDistance(block0, block3)
    hammingDist += hammingDistance(block1, block2)
    hammingDist += hammingDistance(block1, block3)
    hammingDist += hammingDistance(block2, block3)
    hammingDist /= 18
    dists.append((keysize, hammingDist/float(keysize)))
dists = sorted(dists,key=lambda x:x[1])

res=[]
for guessedKeysize in [d[0] for d in dists[0:5]]:
    print "Trying with key size %s"%guessedKeysize
    j=0
    blocks=[]
    while j<len(encoded):
        end = j+guessedKeysize
        if end > len(encoded): 
            end = len(encoded)
        blocks.append(encoded[j:end])
        j+=guessedKeysize
    transposed=[]
    for i in range(guessedKeysize):
        transposedBlock = []
        for block in blocks:
            if i<len(block): transposedBlock.append(block[i])
            else: transposedBlock.append(0)
        transposed.append(transposedBlock)
    key=[]
    for blockNum, block in enumerate(transposed):
        key.append(crackSingleCharXorCipher(block)[0])
    decoded = xorCipher(encoded,key)
    res.append((calcScore(decoded),bytearray(key),decoded))
res = sorted(res,reverse=True)

print 'Encryption key: "%s"\nDecoded:\n%s'%(res[0][1],res[0][2])

37
Trying with key size 8
Trying with key size 29
Trying with key size 15
Trying with key size 39
Trying with key size 2
Encryption key: "Terminator X: Bring the noise"
Decoded:
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -